In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import re



In [ ]:
df = pd.read_json("/content/train.json")
df.head(5)

# df["description"]

df["description"][4]

'Spacious 1 Bedroom 1 Bathroom in Williamsburg!Apartment Features:- Renovated Eat in Kitchen With Dishwasher- Renovated Bathroom- Beautiful Hardwood Floors- Lots of Sunlight- Great Closet Space- Freshly Painted- Heat and Hot Water Included- Live in Super Nearby L, J, M & G Trains !<br /><br />Contact Information:Kenneth BeakExclusive AgentC: 064-692-8838Email: kagglemanager@renthop.com, Text or Email to schedule a private viewing!<br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><p><a  website_redacted '

We can see here that there are html tags in the text which is not required

In [9]:
#Removal of outlier
upper_bound = np.percentile(df["price"].values, 99.9)
df_filtered = df[df["price"] <= upper_bound]

In [10]:
df_filtered["date"] = pd.to_datetime(df_filtered["created"]).dt.date
df_filtered["year"] = pd.to_datetime(df_filtered["date"]).dt.year
df_filtered["month"] = pd.to_datetime(df_filtered["date"]).dt.month
df_filtered["day"] = pd.to_datetime(df_filtered["date"]).dt.day
df_filtered["hour"] = pd.to_datetime(df_filtered["date"]).dt.hour
df_filtered["weekday"] = pd.to_datetime(df_filtered["date"]).dt.weekday
df_filtered["quarter"] = pd.to_datetime(df_filtered["date"]).dt.quarter
df_filtered.head(5)

C:\Users\musab\AppData\Local\Temp\ipykernel_27764\1562200966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["date"] = pd.to_datetime(df_filtered["created"]).dt.date
C:\Users\musab\AppData\Local\Temp\ipykernel_27764\1562200966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["year"] = pd.to_datetime(df_filtered["date"]).dt.year
C:\Users\musab\AppData\Local\Temp\ipykernel_27764\1562200966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,...,price,street_address,interest_level,date,year,month,day,hour,weekday,quarter
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,...,2400,145 Borinquen Place,medium,2016-06-16,2016,6,16,0,3,2
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,...,3800,230 East 44th,low,2016-06-01,2016,6,1,0,2,2
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,...,3495,405 East 56th Street,medium,2016-06-14,2016,6,14,0,1,2
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,...,3000,792 Metropolitan Avenue,medium,2016-06-24,2016,6,24,0,4,2
15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,...,2795,340 East 34th Street,low,2016-06-28,2016,6,28,0,1,2


PREPROCESSING 
- Removal of html tags
- Convert text to lowercase


In [11]:
def remove_html_tags(description):
    description = re.sub(r'<[^>]+>', '', description) 
    description = re.sub(r'\s+', ' ', description)    
    return description.strip()


df_filtered['description'] = df_filtered['description'].apply(remove_html_tags)

df_filtered["description"] = df_filtered["description"].apply(lambda desc: desc.lower())

df_filtered['description'][6]

C:\Users\musab\AppData\Local\Temp\ipykernel_27764\2389221455.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['description'] = df_filtered['description'].apply(remove_html_tags)
C:\Users\musab\AppData\Local\Temp\ipykernel_27764\2389221455.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["description"] = df_filtered["description"].apply(lambda desc: desc.lower())


'brand new gut renovated true 2 bedroomfind yourself and your home in the center of it all. steps from grand central station, at the epicenter of manhattan, the centra combines convenience and luxury to create a perfectly balanced living experience. offering newly renovated over sized apartment layouts.full time doormanelevatornewly renovated hallwayslaundry in buildingon-site parking garagei operate with the utmost care and integrity. the client is my #1 priority. contact me for a viewing of the great apartment, i\'m more than confident we\'ll find a place for you to call home.call/text keon: email: if you require a move within 30 days write "urgent" in the subject email or text message to be taken with high priority.one month free - net effective rent listed<a website_redacted'

Tokenization : Split the text into words (tokens) for easier processing.

In [22]:

import spacy
import pandas as pd

In [23]:
# Load spaCy's pre-trained model (small model for efficiency)
nlp = spacy.load("en_core_web_sm")

# Function to tokenize descriptions
def spaCy_tokenize(description):
    doc = nlp(description)  # Process the description through spaCy
    return [token.text for token in doc]  # Extract tokens from the processed document

# Assuming df_filtered is your DataFrame
# Apply tokenization to the 'description' column
df_filtered['tokenized_description'] = df_filtered['description'].apply(spaCy_tokenize)

# Display the tokenized descriptions
print(df_filtered[['description', 'tokenized_description']].head())


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.